In [1]:
import random
import time

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import torch

from src.model.activation import Relu, LeakyRelu, Linear, Sigmoid
from src.model.base import NeuralNetwork
from src.model.layer import Convolutional3x3x16x0x1, Input3D, Input, FullyConnected, Flatten
from src.model.loss import CategoricalCrossEntropy
from src.model.metric import AccuracyOneHot
from src.model.optimizer import Adam, SGD

In [2]:
# train_data = [
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }
    
# ]

In [3]:
# layers = [
#     Convolutional3x3x16x0x1(LeakyRelu(), kernel=2),
#     Convolutional3x3x16x0x1(LeakyRelu(), kernel=2),
#     Flatten(),
#     #FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input3D((1, 5, 5)),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )


# start_time = time.time()
# epochs = 1
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_111')
# import sys
# sys.exit(0)

In [4]:
def get_onehot_digit(digit):
    output = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    return output[-digit:] + output[:-digit]

In [5]:
def get_digits_data():

    train_dataset = []

    train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

    for index in train.index:
        input_values = [
            float(val) / 255.0 for val in train.loc[index].drop("label").values
        ]
        train_dataset.append(
            {
                "input": input_values,
                "output": get_onehot_digit(int(train.loc[index]["label"])),
            }
        )

    random.seed(1)
    random.shuffle(train_dataset)
    return train_dataset[:400], train_dataset[400:500]

train_dataset, test_dataset = get_digits_data()

layers = [
    FullyConnected(256, LeakyRelu()),
    FullyConnected(128, LeakyRelu()),
    FullyConnected(64, LeakyRelu()),
    FullyConnected(10, Linear()),
]
nn = NeuralNetwork(
    Input(784),
    layers,
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossEntropy(),
    metric=AccuracyOneHot(),
    convert_prediction="argmax",
    on_cuda=False
)

start_time = time.time()

epochs = 10
history = nn.fit(train_dataset, test_dataset, epochs=epochs, batch_size=1, verbose=True)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch: 1/10, train loss: 2.05, train Accuracy: 0.55, test loss: 1.9547, test Accuracy: 0.64, epoch time: 0.671 s
Epoch: 2/10, train loss: 1.7897, train Accuracy: 0.83, test loss: 1.7943, test Accuracy: 0.74, epoch time: 0.78 s
Epoch: 3/10, train loss: 1.6587, train Accuracy: 0.91, test loss: 1.6958, test Accuracy: 0.83, epoch time: 0.779 s
Epoch: 4/10, train loss: 1.5767, train Accuracy: 0.9725, test loss: 1.7187, test Accuracy: 0.82, epoch time: 0.672 s
Epoch: 5/10, train loss: 1.5404, train Accuracy: 0.9775, test loss: 1.7236, test Accuracy: 0.79, epoch time: 0.661 s
Epoch: 6/10, train loss: 1.5183, train Accuracy: 0.99, test loss: 1.8062, test Accuracy: 0.74, epoch time: 0.708 s
Epoch: 7/10, train loss: 1.5053, train Accuracy: 0.995, test loss: 1.6864, test Accuracy: 0.8, epoch time: 0.747 s
Epoch: 8/10, train loss: 1.4905, train Accuracy: 0.9975, test loss: 1.6913, test Accuracy: 0.81, epoch time: 1.383 s
Epoch: 9/10, train loss: 1.4818, train Accuracy: 1.0, test loss: 1.7146, test

In [6]:
def plot_digit(image):
    image = torch.tensor(image).numpy().reshape(28, 28)
    plt.figure()
    plt.imshow(image, cmap="gray")
    plt.axis("off")
    plt.show()

    return

In [7]:
def get_digits_data_cnn():

    train_dataset = []

    train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

    for index in train.index:
        input_values = [
            float(val) / 255.0 for val in train.loc[index].drop("label").values
        ]
        train_dataset.append(
            {
                "input": [torch.tensor(input_values).reshape(28, 28).tolist()],
                "output": get_onehot_digit(int(train.loc[index]["label"])),
            }
        )

    # print(train_dataset[:2])
    # import sys
    # sys.exit(1)
    
    random.seed(1)
    random.shuffle(train_dataset)
    return train_dataset[:400], train_dataset[400:500]

In [8]:
train_data, test_data = get_digits_data_cnn()

layers = [
    Convolutional3x3x16x0x1(LeakyRelu(), filters=4, kernel=3),
    Convolutional3x3x16x0x1(LeakyRelu(), filters=4, kernel=3),
    Convolutional3x3x16x0x1(LeakyRelu(), filters=4, kernel=3),
    #Convolutional3x3x16x0x1(LeakyRelu(), kernel=2),
    Flatten(),
    FullyConnected(256, LeakyRelu()),
    FullyConnected(10, Linear())
]
nn = NeuralNetwork(
    Input3D((1, 28, 28)),
    layers,
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossEntropy(),
    metric=AccuracyOneHot(),
    convert_prediction='argmax',
    on_cuda=False
)

#plot_digit(train_data[0]['input'][0])

start_time = time.time()
epochs = 10
nn.fit(train_data, test_data, epochs=epochs, verbose=True)
print('exec time: ', time.time() - start_time)

print('test_111')

Epoch: 1/10, train loss: 2.0412, train Accuracy: 0.565, test loss: 1.8861, test Accuracy: 0.79, epoch time: 94.741 s
Epoch: 2/10, train loss: 1.7296, train Accuracy: 0.91, test loss: 1.7429, test Accuracy: 0.84, epoch time: 95.952 s
Epoch: 3/10, train loss: 1.61, train Accuracy: 0.985, test loss: 1.8214, test Accuracy: 0.79, epoch time: 95.057 s
Epoch: 4/10, train loss: 1.5484, train Accuracy: 0.9925, test loss: 1.8267, test Accuracy: 0.84, epoch time: 95.92 s
Epoch: 5/10, train loss: 1.507, train Accuracy: 0.9975, test loss: 1.7869, test Accuracy: 0.82, epoch time: 94.766 s
Epoch: 6/10, train loss: 1.4937, train Accuracy: 1.0, test loss: 1.7903, test Accuracy: 0.85, epoch time: 94.791 s
Epoch: 7/10, train loss: 1.4918, train Accuracy: 1.0, test loss: 1.7841, test Accuracy: 0.83, epoch time: 94.735 s
Epoch: 8/10, train loss: 1.4882, train Accuracy: 1.0, test loss: 1.7989, test Accuracy: 0.79, epoch time: 94.923 s
Epoch: 9/10, train loss: 1.501, train Accuracy: 1.0, test loss: 1.8038, t

In [9]:
# train_data = [
#     {
#         'input': [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# layers = [
#     FullyConnected(25, LeakyRelu()),
#     FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input(25),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )

# start_time = time.time()
# epochs = 200
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_222')